## Goal: Run benchmark on stanfordnlp/SHP

In [19]:
from datasets import Dataset, load_dataset

ds = load_dataset("stanfordnlp/SHP")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [24]:
import torch
from torch.utils.data import DataLoader
        
BATCH_SIZE = 32
df = ds['test'].to_pandas()
df = df.groupby("history").apply(
    lambda x: ''.join([
        f"Response {idx}: {response}\n" for idx, response in enumerate(list(x["human_ref_A"]) + list(x["human_ref_B"]))
    ]
)
).reset_index(name="responses")

dataset = Dataset.from_pandas(df)

data_loader = DataLoader(
    dataset, batch_size=BATCH_SIZE
)

/tmp/ipykernel_453/2452845386.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("history").apply(


In [30]:
from openai import OpenAI
from functools import partial

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id

# Round 1
# messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
# response = client.chat.completions.create(model=model, messages=messages)
def response_fn(messages):
    return client.chat.completions.create(model=model, messages=messages)

In [31]:
import copy
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

model_responses = {
    'prompt': [],
    'expert_responses': [],
    'model_responses': []
}
for batch_idx, batch in tqdm(enumerate(data_loader)):
    model_responses['prompt'].extend(batch['history'])
    model_responses['expert_responses'].extend(batch['responses'])
    batched_messages = [
        [{"role": "user", "content": question}]
        for question in batch['history']
    ]
    
    with ThreadPoolExecutor(max_workers=BATCH_SIZE) as executor:
        results = list(executor.map(response_fn, batched_messages))
    model_reasoning = [result.choices[0].message.reasoning_content + result.choices[0].message.content for result in results]
    model_responses['model_responses'].extend(model_reasoning)
    
    break

0it [00:15, ?it/s]


BadRequestError: Error code: 400 - {'object': 'error', 'message': "This model's maximum context length is 1024 tokens. However, you requested 1112 tokens in the messages, Please reduce the length of the messages.", 'type': 'BadRequestError', 'param': None, 'code': 400}